In [1]:
import os
import myServices 
import randForReg as rfr
import numpy as np
import pandas as pd
import sklearn
import joblib

In [2]:
# to compute ececution time do:
# with timeit():
#     # your code, e.g., 
class timeit(): 
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

## Random Forest Section

In [4]:
sklearn.metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_

In [ ]:
DS = pd.read_csv('/Users/abdielfer/DESS/Internship2022/RNCanWork/datasetBasin1.csv')

In [ ]:
DS.isna().sum()

In [ ]:
DS.info()

In [ ]:
DS['percentage'].unique()

In [ ]:
DS['percentage'].value_counts()

In [ ]:
# DS['weights'] = DS['percentage'].map({0:3, 1: 60, 5: 37})
sample = DS.sample(n=10000) # or use : frac=0.05
print(sample.head)
sample['percentage'].unique()

In [ ]:
sample['percentage'] = sample['percentage']*10

In [ ]:
print(sample['percentage'].unique())

In [ ]:
sample['percentage'].value_counts()

In [ ]:
sample.drop(columns=['x_coord', 'y_coord'],inplace = True)
print(sample.head)

In [ ]:
sample.to_csv('./sample.csv')

In [4]:
#Data import and exploration
datasetPath = './sample.csv'
rfReg  = rfr.implementRandomForestRegressor(datasetPath,'percentage', 0.2)
x_train,x_validation,y_train, y_validation = rfReg.getSplitedDataset()

In [ ]:
with timeit():
    bestEstimator = rfReg.fitRFRegressorWeighted(False)

In [3]:
# import executeRFReg as exe
with timeit():
    rfr.main()

 (8000, 7)   : (2000, 7)
Label balance on Training set:  
 0     7824
50     143
10      33
Name: percentage, dtype: int64
Label balance on Validation set:  
 0     1959
50      32
10       9
Name: percentage, dtype: int64
Fitting 3 folds for each of 8160 candidates, totalling 24480 fits
./models/rwReg/rfwgs_2207191740.pkl
      feature  importance
1    disToRiv       0.309
6   elevation       0.155
4        FAcc       0.144
5       slope       0.126
0  Unnamed: 0       0.106
2         TWI       0.091
3         TPI       0.068
The best parameters: {'bootstrap': False, 'max_depth': 16, 'max_features': 0.30000000000000004, 'max_leaf_nodes': None, 'n_estimators': 110}
Mean Absolute Error:  8.028302506573857
Mean Squared Error:  832.3772221696705
Root Mean Squared Error:  28.850948375567665
runtime: 0:34:24.053446
